In [5]:
import ccxt
import os
import datetime

from dotenv import dotenv_values
from pprint import pprint

import ccxt.async_support as ccxt # link against the asynchronous version of ccxt

In [6]:
from functions.BS_pricer import BS_pricer
from functions.Parameters import Option_param
from functions.Processes import Diffusion_process

import numpy as np
import scipy as scp
import scipy.stats as ss
from scipy.integrate import quad
from functools import partial

import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
config = {
    **dotenv_values(".env.shared"),  # load shared development variables
    **dotenv_values(".env.secret"),  # load sensitive variables
    **os.environ,  # override loaded values with environment variables
}

exchange = ccxt.delta(config)
exchange.verbose = False 

exchange.check_required_credentials()
print(exchange.requiredCredentials) 

{'apiKey': True, 'secret': False, 'uid': False, 'login': False, 'password': False, 'twofa': False, 'privateKey': False, 'walletAddress': False, 'token': False}


In [9]:
%%capture --no-display

# markets = await exchange.fetchMarkets() # Full reloading
symbol = "BTC/USDT:USDT"
markets = exchange.load_markets()
balance = exchange.fetch_balance()
currencies = exchange.fetch_currencies()
orderbook = exchange.fetch_order_book(symbol)
l2_orderbook = exchange.fetch_l2_order_book(symbol)
fetchTrades = exchange.fetch_trades(symbol)
ticker = exchange.fetch_ticker(symbol)
positions = exchange.fetch_positions()
ledger = exchange.fetch_ledger()

# risk = exchange.fetch_positions_risk(symbol) # not suppoted yet
# transactions = exchange.fetch_transactions() # not supported yet

In [10]:
delta_exchange_markets = await exchange.load_markets()
# Retrieve only options contract id
_ = {}
for k, v in delta_exchange_markets.items():
    if k.startswith(symbol): _.update({k:v})

btc_options = {}
for k, v in _.items():
    if v["option"]: btc_options.update({k:v})

In [11]:
# to get IV/Greeks
delta_exchange_ticker = await exchange.fetchTickers(list(btc_options.keys()))

# to get Index price (Underlying price)
index_info = await exchange.fetch_ticker(symbol, params={'price':'index'})
index_price = index_info['last']

In [13]:
option_contract = "BTC/USDT:USDT-221129:15800:C"
contract = delta_exchange_ticker[option_contract]
pprint(contract)
a = datetime.datetime.now()
b = datetime.datetime.strptime(contract["expiryDatetime"], "%Y-%m-%dT%H:%M:%SZ")

seconds_until_expiration = b-a
seconds_until_expiration

{'ask': None,
 'askVolume': None,
 'average': 427.0,
 'baseVolume': 0.051000000000000004,
 'bid': None,
 'bidVolume': None,
 'change': 0.0,
 'close': 427.0,
 'datetime': '2022-11-29T00:45:35.122Z',
 'high': 427.0,
 'info': {'close': '427',
          'contract_type': 'call_options',
          'greeks': {'delta': '0.9200',
                     'gamma': '0.0005',
                     'rho': '0.1860',
                     'spot': '16177.515698727313',
                     'theta': '-43.4060',
                     'vega': '0.8615'},
          'high': '427',
          'low': '427',
          'mark_price': '387.2324975',
          'mark_vol': '0.4702156595885753631591796875',
          'oi': '0.0510',
          'oi_contracts': '51',
          'oi_value': '0.0510',
          'oi_value_symbol': 'BTC',
          'oi_value_usd': '823.8548',
          'open': '427',
          'price_band': {'lower_limit': '64.1154418522176200000000000',
                         'upper_limit': '890.30660760'},
    

KeyError: 'expiryDatetime'

In [18]:
"""
    Retrieve
    IV
    index price (Underlying price)
    days until expiration
    strike price
    interest rate
    dividend yield (0)
"""
greeks = contract["info"]["greeks"]
ask_iv = float(contract["info"]["quotes"]['ask_iv'])
ask_size = float(contract["info"]["quotes"]['ask_size'])
best_ask = float(contract["info"]["quotes"]['best_ask'])
best_bid = float(contract["info"]["quotes"]['best_bid'])
bid_iv = float(contract["info"]["quotes"]['bid_iv'])
bid_size = float(contract["info"]["quotes"]['bid_size'])
mark_iv = float(contract["info"]["quotes"]['mark_iv'])
strike_price = float(contract["info"]["strike_price"])
spot_price = float(contract["info"]["spot_price"])

In [25]:
S0=spot_price
K=strike_price
T=1/(365*24)
r=0.1
sigma=ask_iv

In [35]:
def binomial_price(S0, K, T, r, sigma, N=15000, payoff="call"):
    # N number of periods or number of time steps  

    dT = float(T) / N                             # Delta t
    u = np.exp(sigma * np.sqrt(dT))                 # up factor
    d = 1.0 / u                                   # down factor 

    V = np.zeros(N+1)                             # initialize the price vector
    S_T = np.array( [(S0 * u**j * d**(N - j)) for j in range(N + 1)] )  # price S_T at time T

    a = np.exp(r * dT)    # risk free compounded return
    p = (a - d)/ (u - d)  # risk neutral up probability
    q = 1.0 - p           # risk neutral down probability   

    if payoff =="call":
        V[:] = np.maximum(S_T-K, 0.0)
    else:
        V[:] = np.maximum(K-S_T, 0.0)

    for i in range(N-1, -1, -1):
        V[:-1] = np.exp(-r*dT) * (p * V[1:] + q * V[:-1])    # the price vector is overwritten at each step
            
    return V[0]


In [36]:
call_price = BS_pricer.BlackScholes("call", S0=S0, K=K, T=T ,r=r, sigma=sigma)
print(call_price) # our price
print(binomial_price(S0,K,T,r,sigma)) # binomial price
print(best_ask) # market price

377.570052304769
377.57005065889064
398.0
